In [1]:
import pandas as pd

import glob

from pathlib import Path
import csv

In [2]:
## Load pickle for exports and imports of data  
import pickle 
def load_obj(path):
    with open(path, 'rb') as f:
        return pickle.load(f)
    
def save_obj(obj, path ):
    with open(path, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [3]:
ytd_variable = '2020-12' ## At the moment, FTD data goes to Dec 31st 2021. This will give dataframe from Dec 1 2020 to Dec 31st 2021 with FTD data currently 

In [1]:
def do_ftd_calculations(data,symbol):
    
    df = data[symbol]['dataFrame']   #data['dataFrame']
    total_ftds_all = df.QUANTITY_FAILS.sum() ## Add to data['ftd_stats']
    df2021 = df.loc['2021':]
    total_ftds_2021 = df2021.QUANTITY_FAILS.sum() ## Add to data['ftd_stats'] 
    ytd_ftd = df.loc[ytd_variable:]
    total_ftds_ytd = ytd_ftd.QUANTITY_FAILS.sum() ## Add to data['ftd_stats'] 
    outstandingShares = data[symbol]['Fundamentals']['sharesOutstanding']
    float_shares = data[symbol]['Fundamentals']['floatShares']
    
    if outstandingShares != 0:
        try:
            os_ftd_pct_all = total_ftds_all / outstandingShares * 100 ## Add to data['ftd_stats']
            os_ftd_pct_2021 = total_ftds_2021 / outstandingShares * 100 ## Add to data['ftd_stats']
            os_ftd_pct_ytd = total_ftds_ytd / outstandingShares * 100 ## Add to data['ftd_stats']
        except:
            os_ftd_pct_all=0
            os_ftd_pct_2021=0
            os_ftd_pct_ytd=0
    else:
        os_ftd_pct_all=0
        os_ftd_pct_2021=0
        os_ftd_pct_ytd=0
    if float_shares != 0:
        try:
            float_ftd_pct_all = total_ftds_all / float_shares * 100 ## Add to data['ftd_stats']
            float_ftd_pct_2021 = total_ftds_2021 / float_shares * 100 ## Add to data['ftd_stats']
            float_ftd_pct_ytd = total_ftds_ytd / float_shares * 100 ## Add to data['ftd_stats']
        except:
            float_ftd_pct_all=0
            float_ftd_pct_2021=0
            float_ftd_pct_ytd=0
    else:
        float_ftd_pct_all=0
        float_ftd_pct_2021=0
        float_ftd_pct_ytd=0
    
    ## All data 
    totalvolume1 = df.ShortVolumeNSDQ.sum()
    totalvolume2 = df.ShortVolumeNYSE.sum()
    totalshortvolume = totalvolume1+totalvolume2

    totalvolume1 = df.ShortExemptVolumeNSDQ.sum()
    totalvolume2 = df.ShortExemptVolumeNYSE.sum()
    totalshortexemptvolume = totalvolume1+totalvolume2

    totalvolume1 = df.TotalVolumeNSDQ.sum()
    totalvolume2 = df.TotalVolumeNYSE.sum()
    total_volume_nsdq_data= totalvolume1+totalvolume2

    total_volume_fmp_data = df.volume.sum()
    
    if total_volume_fmp_data != 0:
        short_volume_FMPpct_all_data = totalshortvolume / total_volume_fmp_data * 100 ## Add to data['ftd_stats']
    else:
        short_volume_FMPpct_all_data=0
    if total_volume_nsdq_data != 0:
        short_volume_NSDQpct_all_data= totalshortvolume / total_volume_nsdq_data * 100 ## Add to data['ftd_stats']
    else:
        short_volume_NSDQpct_all_data=0
        
    if total_volume_fmp_data != 0:
        shortexempt_volume_FMPpct_all_data = totalshortexemptvolume / total_volume_fmp_data * 100 ## Add to data['ftd_stats']
    else:
        shortexempt_volume_FMPpct_all_data=0
    if total_volume_nsdq_data != 0:
        shortexempt_volume_NSDQpct_all_data= totalshortexemptvolume / total_volume_nsdq_data * 100 ## Add to data['ftd_stats']
    else:
        shortexempt_volume_NSDQpct_all_data=0
        
    ## 1-year data
    totalvolume1 = ytd_ftd.ShortVolumeNSDQ.sum()
    totalvolume2 = ytd_ftd.ShortVolumeNYSE.sum()
    totalshortvolume = totalvolume1+totalvolume2
    totalshortvolume2021 = totalshortvolume

    totalvolume1 = ytd_ftd.ShortExemptVolumeNSDQ.sum()
    totalvolume2 = ytd_ftd.ShortExemptVolumeNYSE.sum()
    totalshortexemptvolume = totalvolume1+totalvolume2
    totalshortexemptvolume2021 = totalshortexemptvolume

    totalvolume1 = ytd_ftd.TotalVolumeNSDQ.sum()
    totalvolume2 = ytd_ftd.TotalVolumeNYSE.sum()
    total_volume_nsdq_data= totalvolume1+totalvolume2

    total_volume_fmp_data = ytd_ftd.volume.sum()

    if total_volume_fmp_data != 0:
        short_volume_FMPpct_ytd = totalshortvolume / total_volume_fmp_data * 100 ## Add to data['ftd_stats']
    else:
        short_volume_FMPpct_ytd=0
    if total_volume_nsdq_data != 0:
        short_volume_NSDQpct_ytd= totalshortvolume / total_volume_nsdq_data * 100 ## Add to data['ftd_stats']
    else: 
        short_volume_NSDQpct_ytd=0
        
    if total_volume_fmp_data != 0: 
        shortexempt_volume_FMPpct_ytd = totalshortexemptvolume / total_volume_fmp_data * 100 ## Add to data['ftd_stats']
    else: 
        shortexempt_volume_FMPpct_ytd=0
    if total_volume_nsdq_data != 0:
        shortexempt_volume_NSDQpct_ytd= totalshortexemptvolume / total_volume_nsdq_data * 100 ## Add to data['ftd_stats']
    else: 
        shortexempt_volume_NSDQpct_ytd=0
    
    ftd_stats = {
        'total_ftds_all':total_ftds_all,
        'total_ftds_2021':total_ftds_2021,
        'total_ftds_ytd':total_ftds_ytd,
        'os_ftd_pct_all':os_ftd_pct_all,
        'os_ftd_pct_2021':os_ftd_pct_2021,
        'os_ftd_pct_ytd':os_ftd_pct_ytd,
        'float_ftd_pct_all':float_ftd_pct_all,
        'float_ftd_pct_2021':float_ftd_pct_2021,
        'float_ftd_pct_ytd':float_ftd_pct_ytd,
        'short_volume_NSDQpct_all_data':short_volume_NSDQpct_all_data,
        'shortexempt_volume_NSDQpct_all_data':shortexempt_volume_NSDQpct_all_data,
        'short_volume_NSDQpct_ytd':short_volume_NSDQpct_ytd,
        'shortexempt_volume_NSDQpct_ytd':shortexempt_volume_NSDQpct_ytd,
        'short_volume_FMPpct_all_data':short_volume_FMPpct_all_data,
        'shortexempt_volume_FMPpct_all_data':shortexempt_volume_FMPpct_all_data,
        'short_volume_FMPpct_ytd':short_volume_FMPpct_ytd,
        'shortexempt_volume_FMPpct_ytd':shortexempt_volume_FMPpct_ytd
    }
    
    ## Add to data passed to function, then return 
    data[symbol]['ftd_stats'] = ftd_stats
    return data 

In [5]:
import_path = '../Resources/03_datadict_success_list.pkl'
datadict_success_list = load_obj(import_path)

len(datadict_success_list)

5493

In [6]:
for symbol in datadict_success_list:
    import_path = '../FilesExport_DataDicts/'+symbol+'_data_dict.pkl'
    export_path = '../FilesExport_Complete_DataDicts/'+symbol+'_data_dict.pkl'
    
    data = load_obj(import_path)    
    data = do_ftd_calculations(data,symbol)
    
    save_obj(data,export_path)